In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName('CalcPI_Accum').setMaster('yarn')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 21:38:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/12 21:38:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/12 21:38:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/12 21:38:38 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/12 21:38:38 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/11/12 21:38:38 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/11/12 21:38:38 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
24/11/12 21:38:39 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.


app_id                                   application_1727681258360_0027


In [4]:
from pyspark import Accumulator
import random
import math
 
accum = spark.sparkContext.accumulator(0)
 
def get_value():
    return 1 if math.sqrt(math.pow(random.random(), 2) + math.pow(random.random(), 2)) <1 else 0
 
def func(iterator):
    global accum    
    random.seed()
   
    for i in iterator:
        accum += get_value()

In [7]:
length = 100000000
part_num = 1000
rdd = spark.sparkContext.parallelize(range(0, length), part_num)
 
accum.value = 0
rdd.foreachPartition(func)
print('Pi value approx = ', accum.value/(length*0.25))

Pi value approx =  3.14159612


In [8]:
spark.stop()